En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [1]:
file_path = "farmers-protest-tweets-2021-2-4.json"

In [2]:
%load_ext line_profiler
%load_ext memory_profiler

# Pregunta 1

Se tomaron dos enfoques distintos. El primero consistió en leer los datos con la librería JSON y luego trabajar la información con Pandas. Para el segundo enfoque se utilizó PySpark para la lectura y tratamiento de los datos.

En cuanto al tiempo de ejecución con el primer método, se observó que los tiempos eran estables y repetibles, con una duración de alrededor de 11 segundos. En cambio, el segundo método dependía de forma sustancial del estado de la sesión de Spark. En la primera ejecución consecutiva, los tiempos de ejecución rondaban los 15 segundos, muy distintos a los tiempos de las ejecuciones posteriores, que rondaban entre 5 y 6 segundos.

Debido a esto, se tomó la decisión de elegir el primer método para q1_time, únicamente por su estabilidad. Cabe considerar que, en el caso de tener un ambiente de ejecución más estable, el segundo método podría ofrecer mejores resultados a largo plazo.

En cuanto al uso de memoria, este fue un análisis más directo. Con el método 1, siempre se observó un uso de hasta 1300 MiB, muy por encima del método 2, que rondaba los 300 MiB de forma constante. Por ende, se seleccionó el método 2 para q1_memory

## Metodo 1

Se ejecuto multiples veces y siempre se consigui un tiempo de ejecucion de alrededor de 11s

In [3]:
from q1_time import  q1_time

In [4]:
%lprun -f q1_time q1_time(file_path=file_path)

Timer unit: 1e-09 s

Total time: 11.9141 s
File: /home/sandbox/files/latam-challenge/src/q1_time.py
Function: q1_time at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
     9                                           
    10         1       1803.0   1803.0      0.0      def read_json(file_path):
    11                                                   data = []
    12                                                   with open(file_path, "r") as f:
    13                                                       # En el archivo hay un json por linea, hay que itearr y extrar cada uno
    14                                                       for line in f:
    15                                                           data.append(json.loads(line))
    16                                                       f.close()
    17                             

In [5]:
%lprun -f q1_time q1_time(file_path=file_path)

Timer unit: 1e-09 s

Total time: 10.4266 s
File: /home/sandbox/files/latam-challenge/src/q1_time.py
Function: q1_time at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
     9                                           
    10         1       1262.0   1262.0      0.0      def read_json(file_path):
    11                                                   data = []
    12                                                   with open(file_path, "r") as f:
    13                                                       # En el archivo hay un json por linea, hay que itearr y extrar cada uno
    14                                                       for line in f:
    15                                                           data.append(json.loads(line))
    16                                                       f.close()
    17                             

In [6]:
%mprun -f q1_time q1_time(file_path=file_path)

Filename: /home/sandbox/files/latam-challenge/src/q1_time.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     8    320.5 MiB    320.5 MiB           1   def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
     9                                         
    10    320.5 MiB      0.0 MiB           2       def read_json(file_path):
    11    320.5 MiB      0.0 MiB           1           data = []
    12   1238.1 MiB      0.0 MiB           2           with open(file_path, "r") as f:
    13                                                     # En el archivo hay un json por linea, hay que itearr y extrar cada uno
    14   1238.1 MiB     20.9 MiB      117408               for line in f:
    15   1238.1 MiB    896.7 MiB      117407                   data.append(json.loads(line))
    16   1238.1 MiB      0.0 MiB           1               f.close()
    17                                         
    18   1238.1 MiB      0.0 MiB           1           return data
    19

Mem pregunta 1

In [7]:
from q1_memory import  q1_memory

In [8]:
%lprun -f q1_memory q1_memory(file_path=file_path)

your 131072x1 screen size is bogus. expect trouble
24/10/14 23:56:46 WARN Utils: Your hostname, pescara-pc resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/14 23:56:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/14 23:56:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Timer unit: 1e-09 s

Total time: 16.0383 s
File: /home/sandbox/files/latam-challenge/src/q1_memory.py
Function: q1_memory at line 44

Line #      Hits         Time  Per Hit   % Time  Line Contents
    44                                           def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    45                                           
    46         1 3573701868.0    4e+09     22.3      spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
    47                                           
    48         1 5591936868.0    6e+09     34.9      data = spark.read.json(file_path)
    49         1  614141819.0    6e+08      3.8      dataframe = get_dates_and_users(spark, data)
    50         1       1603.0   1603.0      0.0      del data
    51                                           
    52         2      20819.0  10409.5      0.0      output = [
    53                                                   tuple(row)
    54         1 5271904276.0    5e+09     32

In [9]:
%lprun -f q1_memory q1_memory(file_path=file_path)

Timer unit: 1e-09 s

Total time: 5.33392 s
File: /home/sandbox/files/latam-challenge/src/q1_memory.py
Function: q1_memory at line 44

Line #      Hits         Time  Per Hit   % Time  Line Contents
    44                                           def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    45                                           
    46         1  108041663.0    1e+08      2.0      spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
    47                                           
    48         1  919476300.0    9e+08     17.2      data = spark.read.json(file_path)
    49         1  143086115.0    1e+08      2.7      dataframe = get_dates_and_users(spark, data)
    50         1        481.0    481.0      0.0      del data
    51                                           
    52         2      10129.0   5064.5      0.0      output = [
    53                                                   tuple(row)
    54         1 3165196143.0    3e+09     59

In [10]:
%mprun -f q1_memory q1_memory(file_path=file_path)

Filename: /home/sandbox/files/latam-challenge/src/q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    44    307.9 MiB    307.9 MiB           1   def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    45                                         
    46    307.9 MiB      0.0 MiB           1       spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
    47                                         
    48    307.9 MiB      0.0 MiB           1       data = spark.read.json(file_path)
    49    307.9 MiB      0.0 MiB           1       dataframe = get_dates_and_users(spark, data)
    50    307.9 MiB      0.0 MiB           1       del data
    51                                         
    52    307.9 MiB      0.0 MiB          24       output = [
    53    307.9 MiB      0.0 MiB          10           tuple(row)
    54    307.9 MiB      0.0 MiB          11           for row in dataframe.select([col("date_part"), col("username")]).collect()
    55

In [19]:
q1_time(file_path=file_path)

[(Date(2021, 2, 12), 'RanbirS00614606'),
 (Date(2021, 2, 13), 'MaanDee08215437'),
 (Date(2021, 2, 17), 'RaaJVinderkaur'),
 (Date(2021, 2, 16), 'jot__b'),
 (Date(2021, 2, 14), 'rebelpacifist'),
 (Date(2021, 2, 18), 'neetuanjle_nitu'),
 (Date(2021, 2, 15), 'jot__b'),
 (Date(2021, 2, 20), 'MangalJ23056160'),
 (Date(2021, 2, 23), 'Surrypuria'),
 (Date(2021, 2, 19), 'Preetm91')]

In [20]:
q1_memory(file_path=file_path)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

# Pregunta 2

Para este problema se tomó un enfoque similar al de la pregunta anterior, pero a diferencia de esta, para el tiempo se utilizó PySpark y para la memoria, funciones convencionales.

Dado que solo es necesario contar ocurrencias a nivel de registro, se puede iterar fila por fila.

Lamentablemente, este enfoque no obtuvo los resultados esperados, y la metodología sin PySpark utilizó prácticamente la misma cantidad de memoria que la que usaba PySpark, e incluso más en algunas pruebas.

In [11]:
from q2_time import  q2_time

%lprun -f q2_time q2_time(file_path=file_path)

Timer unit: 1e-09 s

Total time: 7.70576 s
File: /home/sandbox/files/latam-challenge/src/q2_time.py
Function: q2_time at line 44

Line #      Hits         Time  Per Hit   % Time  Line Contents
    44                                           def q2_time(file_path: str) -> List[Tuple[str, int]]:
    45         1   84369551.0    8e+07      1.1      spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
    46                                           
    47         1  710520069.0    7e+08      9.2      data = spark.read.json(file_path)
    48         1   82613058.0    8e+07      1.1      dataframe = get_most_used_emojis(spark, data)
    49                                           
    50         1        872.0    872.0      0.0      del data
    51                                           
    52         2      11281.0   5640.5      0.0      output = [
    53         1 5829209848.0    6e+09     75.6          tuple(row) for row in dataframe.select([col("emoji"), col("count")]).co

In [12]:
%mprun -f q2_time q2_time(file_path=file_path)

Filename: /home/sandbox/files/latam-challenge/src/q2_time.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    44    308.1 MiB    308.1 MiB           1   def q2_time(file_path: str) -> List[Tuple[str, int]]:
    45    308.1 MiB      0.0 MiB           1       spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
    46                                         
    47    308.1 MiB      0.0 MiB           1       data = spark.read.json(file_path)
    48    308.1 MiB      0.0 MiB           1       dataframe = get_most_used_emojis(spark, data)
    49                                         
    50    308.1 MiB      0.0 MiB           1       del data
    51                                         
    52    308.1 MiB      0.0 MiB          24       output = [
    53    308.1 MiB      0.0 MiB          11           tuple(row) for row in dataframe.select([col("emoji"), col("count")]).collect()
    54                                             ]
    55                      

In [13]:
from q2_memory import  q2_memory

%lprun -f q2_memory q2_memory(file_path)

Timer unit: 1e-09 s

Total time: 362.589 s
File: /home/sandbox/files/latam-challenge/src/q2_memory.py
Function: q2_memory at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def q2_memory(file_path: str) -> List[Tuple[str, int]]:
     8         1       1432.0   1432.0      0.0      result = dict()
     9         2      53550.0  26775.0      0.0      with open(file_path, "r") as f:
    10                                                   # En el archivo hay un json por linea, hay que itearr y extrar cada uno
    11    117408 2009027070.0  17111.5      0.6          for line in f:
    12    117407        3e+11    3e+06     82.9              data = pd.json_normalize(json.loads(line))
    13    234814        1e+10  47962.0      3.1              for c in data.content.to_list():
    14    160329        5e+10 302821.0     13.4                  for e in emoji.emoji_list(c):
    15     42922   42970470.0   1001.1      0.0    

In [14]:
%mprun -f q2_memory q2_memory(file_path)

Filename: /home/sandbox/files/latam-challenge/src/q2_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     7    308.1 MiB    308.1 MiB           1   def q2_memory(file_path: str) -> List[Tuple[str, int]]:
     8    308.1 MiB      0.0 MiB           1       result = dict()
     9    308.1 MiB      0.0 MiB           2       with open(file_path, "r") as f:
    10                                                 # En el archivo hay un json por linea, hay que itearr y extrar cada uno
    11    308.1 MiB      0.0 MiB      117408           for line in f:
    12    308.1 MiB      0.0 MiB      117407               data = pd.json_normalize(json.loads(line))
    13    308.1 MiB      0.0 MiB      234814               for c in data.content.to_list():
    14    308.1 MiB      0.0 MiB      160329                   for e in emoji.emoji_list(c):
    15    308.1 MiB      0.0 MiB       42922                       result[e["emoji"]] = result.get(e["emoji"], 0) + 1
    16             

In [21]:
q2_time(file_path=file_path)

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

In [22]:
q2_memory(file_path)

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

# Pregunta 3

Problemática similar al ejercicio anterior, salvo por la diferencia de que el dato venía dentro de una lista de JSONs y no en un texto plano (exceptuando el hecho de que se tenían que extraer emojis).

Por esta razón, se tomó un enfoque similar al de la pregunta anterior, con resultados similares.

In [15]:
from q3_time import  q3_time

%lprun -f q3_time q3_time(file_path=file_path)

Timer unit: 1e-09 s

Total time: 2.09702 s
File: /home/sandbox/files/latam-challenge/src/q3_time.py
Function: q3_time at line 25

Line #      Hits         Time  Per Hit   % Time  Line Contents
    25                                           def q3_time(file_path: str) -> List[Tuple[str, int]]:
    26         1  109873134.0    1e+08      5.2      spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
    27                                           
    28         1  691153959.0    7e+08     33.0      data = spark.read.json(file_path)
    29         1   40230284.0    4e+07      1.9      dataframe = get_most_mentions(spark, data)
    30                                           
    31         1        722.0    722.0      0.0      del data
    32                                           
    33         2      12412.0   6206.0      0.0      output = [
    34         1 1011795270.0    1e+09     48.2          tuple(row) for row in dataframe.select([col("username"), col("count")]).co

In [16]:
%mprun -f q3_time q3_time(file_path=file_path)

Filename: /home/sandbox/files/latam-challenge/src/q3_time.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    25    308.1 MiB    308.1 MiB           1   def q3_time(file_path: str) -> List[Tuple[str, int]]:
    26    308.1 MiB      0.0 MiB           1       spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
    27                                         
    28    308.1 MiB      0.0 MiB           1       data = spark.read.json(file_path)
    29    308.1 MiB      0.0 MiB           1       dataframe = get_most_mentions(spark, data)
    30                                         
    31    308.1 MiB      0.0 MiB           1       del data
    32                                         
    33    308.1 MiB      0.0 MiB          24       output = [
    34    308.1 MiB      0.0 MiB          11           tuple(row) for row in dataframe.select([col("username"), col("count")]).collect()
    35                                             ]
    36                      

In [17]:
from q3_memory import  q3_memory

%lprun -f q3_memory q3_memory(file_path)

Timer unit: 1e-09 s

Total time: 307.037 s
File: /home/sandbox/files/latam-challenge/src/q3_memory.py
Function: q3_memory at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def q3_memory(file_path: str) -> List[Tuple[str, int]]:
     7         1       2164.0   2164.0      0.0      result = dict()
     8         2     136466.0  68233.0      0.0      with open(file_path, "r") as f:
     9                                                   # En el archivo hay un json por linea, hay que itearr y extrar cada uno
    10    117408 1815098362.0  15459.8      0.6          for line in f:
    11    117407        3e+11    3e+06     97.3              data = pd.json_normalize(json.loads(line))
    12    117407 3005368140.0  25597.9      1.0              if "quotedTweet.mentionedUsers" in data.columns: # Hay registros que no tienen la columna
    13     41436 3248181457.0  78390.3      1.1                  if data["quotedTweet.men

In [18]:
%mprun -f q3_memory q3_memory(file_path)

Filename: /home/sandbox/files/latam-challenge/src/q3_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     6    308.2 MiB    308.2 MiB           1   def q3_memory(file_path: str) -> List[Tuple[str, int]]:
     7    308.2 MiB      0.0 MiB           1       result = dict()
     8    314.0 MiB      0.0 MiB           2       with open(file_path, "r") as f:
     9                                                 # En el archivo hay un json por linea, hay que itearr y extrar cada uno
    10    314.0 MiB      0.0 MiB      117408           for line in f:
    11    314.0 MiB      5.7 MiB      117407               data = pd.json_normalize(json.loads(line))
    12    314.0 MiB      0.0 MiB      117407               if "quotedTweet.mentionedUsers" in data.columns: # Hay registros que no tienen la columna
    13    314.0 MiB      0.0 MiB       41436                   if data["quotedTweet.mentionedUsers"][0]: # Y otros que vienen con "None"
    14    314.0 MiB      0.0 MiB    

In [23]:
q3_time(file_path=file_path)

[('narendramodi', 358),
 ('DelhiPolice', 320),
 ('nytimes', 312),
 ('GretaThunberg', 226),
 ('Kisanektamorcha', 205),
 ('jazzyb', 204),
 ('RakeshTikaitBKU', 204),
 ('mujerxsrising', 173),
 ('Bkuektaugrahan', 166),
 ('sushant_says', 157)]

In [24]:
q3_memory(file_path)

[('narendramodi', 358),
 ('DelhiPolice', 320),
 ('nytimes', 312),
 ('GretaThunberg', 226),
 ('Kisanektamorcha', 205),
 ('RakeshTikaitBKU', 204),
 ('jazzyb', 204),
 ('mujerxsrising', 173),
 ('Bkuektaugrahan', 166),
 ('sushant_says', 157)]